In [9]:
from datetime import datetime, timedelta
from selenium import webdriver
from selenium.common.exceptions import *
from selenium.webdriver.common.by import By

import json
import re
import sys

## Selector class

In [11]:
datetime.today() + timedelta(-25)

datetime.datetime(2019, 2, 22, 22, 19, 43, 737395)

## Main scraper class

In [ ]:
class Scraper:
    """
    Main scraper class.
    """
    
    def __init__(self, metadata, driver = None):
        self.__metadata = metadata
        self.__driver   = driver
        if driver is None:
            try:
                self.__driver = webdriver.Firefox(executable_path = "../driver/geckodriver")
            except WebDriverException as ex:
                self.__close_driver("Error creating Selenium driver", ex)
    
    def __close_driver(self):
        try:
            self.__driver.close()
        except AttributeError:
            pass # 